<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [17]:
def printObj():
    print('''
    Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling:
        Fit a Gensim LDA topic model on Amazon Reviews
        Select appropriate number of topics
        Create some dope visualization of the topics
        Write a few bullets on your findings in markdown at the end
        Note: You don't have to use generators for this assignment
    ''')

In [11]:
prtToDo()


    Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling:
        Fit a Gensim LDA topic model on Amazon Reviews
        Select appropriate number of topics
        Create some dope visualization of the topics
        Write a few bullets on your findings in markdown at the end
        Note: You don't have to use generators for this assignment
    


In [28]:
import pandas as pd
import numpy as np

import os, re, time

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis.gensim

In [4]:
path = '../module1-text-data/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv'
df = pd.read_csv(path)

In [6]:
print(len(df))
df.head()

28332


,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [15]:
# STOPWORDS = set(STOPWORDS).union(set([  ____list words to keep!!!___ ]))

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

# def get_lemmas(text):

#     lemmas = []

#     doc = nlp(text)

#     for token in doc:
#         if (token.text.lower() not in stopwords):
#             if (token.is_stop == False) and (token.is_punct == False) and (token.pos_!= 'PRON'):
#                 lemmas.append(token.lemma_)

#     return lemmas

df['lemz'] = df['reviews.text'].apply(tokenize)
df.head(3)

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs,lemz
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht...","[order, item, bad, quality, missing, backup, s..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht...","[bulk, expensive, way, products, like]"
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht...","[duracell, price, happy]"


In [20]:
# Gensim Model Steps:

#   1. Learn id2word
id2word = corpora.Dictionary(df['lemz'])

#   2. Create bag of words representation (gensim version of count vectorizer)
corpus = [id2word.doc2bow(text) for text in df['lemz']]

In [21]:
#   3. Pass both to LDA model
t0 = time.time()
lda = LdaMulticore(corpus = corpus,
                   id2word = id2word,
                   random_state = 8675309,
                   num_topics = 15, # ---> after corpus, this is most important!
                   passes = 10,
                   workers = 8
                  )
train_time = time.time() - t0
print("train time: %0.3fs" % train_time)

train time: 84.664s


In [24]:
lda.print_topics(5)

[(4,
  '0.050*"tablet" + 0.023*"great" + 0.020*"use" + 0.018*"apps" + 0.018*"amazon" + 0.015*"games" + 0.013*"play" + 0.012*"good" + 0.012*"kindle" + 0.011*"books"'),
 (7,
  '0.024*"battery" + 0.020*"love" + 0.015*"years" + 0.015*"batteries" + 0.014*"day" + 0.013*"kindle" + 0.011*"use" + 0.010*"ve" + 0.009*"new" + 0.008*"kindles"'),
 (10,
  '0.053*"tablet" + 0.043*"old" + 0.039*"loves" + 0.035*"year" + 0.026*"great" + 0.018*"bought" + 0.018*"games" + 0.017*"daughter" + 0.016*"easy" + 0.014*"son"'),
 (12,
  '0.107*"great" + 0.054*"price" + 0.050*"product" + 0.035*"value" + 0.030*"good" + 0.021*"battery" + 0.017*"charge" + 0.015*"excellent" + 0.014*"fast" + 0.011*"life"'),
 (11,
  '0.056*"kids" + 0.048*"love" + 0.032*"tablet" + 0.023*"good" + 0.022*"use" + 0.020*"easy" + 0.019*"great" + 0.019*"games" + 0.016*"apps" + 0.013*"play"')]

In [26]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]
topics = [' '.join(t[0:5]) for t in words]

In [27]:
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
kindle like easy time screen

------ Topic 1 ------
great tablet love use easy

------ Topic 2 ------
easy size kindle use great

------ Topic 3 ------
batteries long great price work

------ Topic 4 ------
tablet great use apps amazon

------ Topic 5 ------
good far great batteries buy

------ Topic 6 ------
batteries good battery quality amazon

------ Topic 7 ------
battery love years batteries day

------ Topic 8 ------
bought loves gift christmas kindle

------ Topic 9 ------
tap echo good alexa like

------ Topic 10 ------
tablet old loves year great

------ Topic 11 ------
kids love tablet good use

------ Topic 12 ------
great price product value good

------ Topic 13 ------
batteries price amazon good great

------ Topic 14 ------
long amazon fine battery device



In [29]:
pyLDAvis.enable_notebook()

In [44]:
my_davies = pyLDAvis.gensim.prepare(lda, corpus, id2word)

C:\Users\RevL\.conda\envs\U4-S1-NLP\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [46]:
type(my_davies)

pyLDAvis._prepare.PreparedData

In [47]:
my_davies

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.215226 -0.055561       1        1  15.163721
4     -0.089508  0.041595       2        1  10.481591
10    -0.188779 -0.087446       3        1   9.855688
13     0.127391 -0.028519       4        1   7.443604
8     -0.134601 -0.007441       5        1   6.547907
0     -0.061640  0.096973       6        1   6.534282
11    -0.166973 -0.080396       7        1   6.444358
1     -0.096547 -0.046644       8        1   5.811070
2     -0.091635  0.051857       9        1   5.799169
12     0.092278 -0.100500      10        1   5.594106
5      0.103443 -0.133344      11        1   4.705317
14     0.076623  0.070014      12        1   4.431250
9      0.012770  0.179121      13        1   4.208557
6      0.119359  0.047292      14        1   3.754089
7      0.082594  0.053000      15        1   3.225296, topic_info=     Category         Freq         Term        Total  loglift  logprob
19    Default  7326.000000    batteries  7326.000000  30.0000  30.0000
22    Default  9000.000000        great  9000.000000  29.0000  29.0000
4197  Default  6819.000000       tablet  6819.000000  28.0000  28.0000
35    Default  5458.000000         good  5458.000000  27.0000  27.0000
2616  Default  2143.000000        loves  2143.000000  26.0000  26.0000
202   Default  2448.000000         kids  2448.000000  25.0000  25.0000
18    Default  4966.000000        price  4966.000000  24.0000  24.0000
174   Default  2003.000000      product  2003.000000  23.0000  23.0000
93    Default  3471.000000         love  3471.000000  22.0000  22.0000
605   Default  2086.000000          old  2086.000000  21.0000  21.0000
3157  Default  3023.000000       kindle  3023.000000  20.0000  20.0000
24    Default  2416.000000         long  2416.000000  19.0000  19.0000
902   Default  2934.000000         easy  2934.000000  18.0000  18.0000
108   Default  1747.000000         year  1747.000000  17.0000  17.0000
205   Default  1176.000000          far  1176.000000  16.0000  16.0000
446   Default  1029.000000        value  1029.000000  15.0000  15.0000
29    Default  3123.000000       bought  3123.000000  14.0000  14.0000
1059  Default  1053.000000         gift  1053.000000  13.0000  13.0000
3     Default  2053.000000      battery  2053.000000  12.0000  12.0000
134   Default  1005.000000         size  1005.000000  11.0000  11.0000
772   Default  1689.000000        games  1689.000000  10.0000  10.0000
10    Default  2218.000000         work  2218.000000   9.0000   9.0000
50    Default  1108.000000      perfect  1108.000000   8.0000   8.0000
502   Default   807.000000         deal   807.000000   7.0000   7.0000
21    Default  1286.000000        brand  1286.000000   6.0000   6.0000
1145  Default  1204.000000      reading  1204.000000   5.0000   5.0000
76    Default   898.000000       brands   898.000000   4.0000   4.0000
32    Default   872.000000    christmas   872.000000   3.0000   3.0000
1615  Default   938.000000     daughter   938.000000   2.0000   2.0000
2050  Default  1806.000000         apps  1806.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
1547  Topic15    35.975903         goes   112.217148   2.2966  -5.6565
4895  Topic15    24.011826        newer    60.669468   2.5073  -6.0608
6959  Topic15    13.670802  transferred    24.761026   2.8401  -6.6241
723   Topic15    39.643559    impressed   140.951431   2.1657  -5.5594
194   Topic15    47.690517        weeks   192.900452   2.0367  -5.3746
105   Topic15    53.111362          job   231.807114   1.9606  -5.2670
5996  Topic15    31.808350     upgraded   101.221901   2.2766  -5.7796
2102  Topic15    56.829716        older   304.263794   1.7563  -5.1993
1277  Topic15    24.909578         knew    71.708748   2.3768  -6.0241
93    Topic15   210.924286         love  3471.571533   0.6333  -3.8879
177   Topic15    98.044495           ve   886.206116   1.

In [48]:
from gensim.models.coherencemodel import CoherenceModel

def compute_coherence_values(dictionary, corpus, limit, start=2, step=3, passes=5):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    limit : Max num of topics
    passes: the number of times the entire lda model & coherence values are calculated

    Returns:
    -------
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    
    coherence_values = []
    
    for iter_ in range(passes):
        for num_topics in range(start, limit, step):
            model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, workers=4)
            coherencemodel = CoherenceModel(model=model,dictionary=dictionary,corpus=corpus, coherence='u_mass')
            coherence_values.append({'pass': iter_, 
                                     'num_topics': num_topics, 
                                     'coherence_score': coherencemodel.get_coherence()
                                    })

    return coherence_values

In [49]:
coherence_values = compute_coherence_values(dictionary=id2word, 
                                                        corpus=corpus,
                                                        start=2, 
                                                        limit=40, 
                                                        step=6,
                                                        passes=40)

In [0]:
topic_coherence = pd.DataFrame.from_records(coherence_values)

In [19]:
printObj()


    Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling:
        Fit a Gensim LDA topic model on Amazon Reviews
        Select appropriate number of topics
        Create some dope visualization of the topics
        Write a few bullets on your findings in markdown at the end
        Note: You don't have to use generators for this assignment
    


## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling